In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout,Lambda
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam,SGD
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import LeakyReLU



In [10]:
im_width = 128
im_height = 128
border = 5
path_train_input = '/content/drive/My Drive/Ag-Net-Dataset-master/input/'
path_train_output = '/content/drive/My Drive/Ag-Net-Dataset-master/target/'

#ids_in=[]
#ids_out=[]


ids = next(os.walk(path_train_output))[2]
print(len(ids))

#for id_ in ids:
  #for i in range(1,8):
    #ids_in.append(path_train_input + 'lc8' + id_[3:-4] + '_' + str(i) + '.tif')
    #ids_out.append(path_train_output + id_)
    

from random import shuffle

#c = list(zip(ids_in, ids_out))

shuffle(ids)

#ids_in,ids_out=zip(*c)

train_ids = ids[0:1500]
valid_ids = ids[1500:]

#valid_ids_out = ids_out[0:int(0.15*len(ids_out))]
#train_ids_out = ids_out[int(0.15*len(ids_out)):]""""

print(len(train_ids))
    


1593
1500


In [0]:

    
def get_data(ids,batch_size):
  while True:
    ids_batches = [ids[i:min(i+batch_size,len(ids))] for i in range(0, len(ids), batch_size)] 
    #ids_out_batches = [ids_out[j:min(j+batch_size,len(ids_out))] for j in range(0, len(ids_out), batch_size)]
    # Load images
    for b in range(len(ids_batches)):
      #print(b)
      #print(":")
      #print(ids_batches[b])
      k=-1
      X = np.zeros((len(ids_batches[b]), im_height, im_width, 7), dtype=np.float32)
      y = np.zeros((len(ids_batches[b]), im_height*im_width, 255), dtype=np.float32)
      for c in range(len(ids_batches[b])):
        k=k+1
        #temp1=np.zeros((1,7),dtype=np.float32)
        for r in range(1,8):
          img = load_img(path_train_input + 'lc8' + ids_batches[b][c][3:-4] + '_' + str(r) + '.tif', color_mode="grayscale")
          x_img = img_to_array(img)
          x_img = resize(x_img, (128, 128), mode='constant', preserve_range=True)
          for p in range(128):
            for q in range(128):
              #print(x_img[p][q]/255)
              X[k][p][q][r-1]=x_img[p][q]/255
          
        #k=k+1
        # Save images
        #X[k, ..., 0] = temp1 / 255  

        # Load masks

        mask = img_to_array(load_img(path_train_output+ids_batches[b][c], color_mode="grayscale"))
        mask = resize(mask, (128, 128), mode='constant', preserve_range=True)
        
        inc=-1
        for p in range(128):
          for q in range(128):
            num=int(mask[p][q])
            temp=np.zeros((255), dtype=np.float32)
            temp[num]=1
            inc=inc+1
            y[k][inc]=temp
            print

      yield X,y          

  

    
#, kernel_initializer="he_normal"


In [0]:
def conv2d_block(input_tensor, n_filters, kernel_size=3,strides=1,batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size),strides=(strides, strides),padding="same", kernel_initializer="he_normal")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
#     # second layer
#     x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size),strides=(strides, strides),padding="same", kernel_initializer="he_normal")(x)
#     if batchnorm:
#         x = BatchNormalization()(x)
#     x = LeakyReLU(alpha=0.1)(x)
#     #x = MaxPooling2D((2, 2)) (x)
# #     p2 = Dropout(rate=dropout)(p2)
    return x
  

def encoder_decoder(x1,ni, kernel_size=3, batchnorm=True,times=None):
  x=GlobalMaxPooling2D()(x1)
  x=Reshape(target_shape=(1,1,times))(x)
  x=Conv2D(filters=ni//2, kernel_size=(kernel_size, kernel_size),padding="same", kernel_initializer="he_normal")(x)
  x=LeakyReLU(alpha=0.1)(x)
  x=Conv2D(filters=ni, kernel_size=(kernel_size, kernel_size),padding="same", kernel_initializer="he_normal")(x)
  x=Activation('sigmoid')(x)
  
  x2=Conv2D(filters=ni, kernel_size=(kernel_size, kernel_size),padding="same", kernel_initializer="he_normal")(x1)
  x2=Activation('sigmoid')(x2)
  
  
  x11=Multiply()([x1,x2])
  x12=Multiply()([x1,x])
  x13=Add()([x11,x12])
  
  return x13
  
  
 
  
  
def DownBlock(x,ni,nf, kernel_size=3, batchnorm=True,down=None):
  inp=x
  x=conv2d_block(x,nf,3,2)
  x=conv2d_block(x,nf,3)
  x=Add()([x,conv2d_block(inp,nf,3,2)])
  if down is not None:
    return encoder_decoder(x,nf, kernel_size=3, batchnorm=True,times=128)
  else:
    return x
  
def UpBlock(down,cross,ni,nf, kernel_size=3, batchnorm=True,down1=None):
  x=Conv2DTranspose(filters=nf, kernel_size=(3, 3),strides=(2,2),padding="same", kernel_initializer="he_normal")(down)
  print(x)
  print(cross)
  x=concatenate([x,cross])
  x=conv2d_block(x,nf,3)
  if down1 is not None:
    return encoder_decoder(x,nf, kernel_size=3, batchnorm=True,times=256)
  else:
    return x
  
  
    

In [0]:
from keras.activations import softmax
from keras.layers.core import Layer, Dense, Dropout, Activation, Flatten, Reshape, Permute
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout,GlobalMaxPooling2D
from keras.layers import Add,Multiply
from keras import backend as K





#print(y_train)


In [0]:
def get_unet(input_img, n_filters=128, dropout=0.15, batchnorm=True):
#     # contracting path
#     c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
#     p1 = MaxPooling2D((2, 2)) (c1)
#     p1 = Dropout(rate=dropout*0.5)(p1)
    
#     c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
#     p2 = MaxPooling2D((2, 2)) (c2)
#     p2 = Dropout(rate=dropout)(p2)


    
#     c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    
#     # expansive path
#     u4 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c3)
#     u4 = concatenate([u4, c2])
#     u4 = Dropout(rate=dropout)(u4)
#     c4 = conv2d_block(u4, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    
#     u5 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c4)
#     u5 = concatenate([u5, c1])
#     u5 = Dropout(rate=dropout)(u5)
#     c5 = conv2d_block(u5, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)

    
#     c6 = conv2d_block(c5, n_filters=255, kernel_size=1, batchnorm=batchnorm)
    
#     c7 = core.Reshape((128*128,255))(c6)
    

#     #Conv2D(255, (1, 1), activation='softmax') (c4)
    
#     outputs = core.Activation('softmax')(c7)
    
#     #outputs=Lambda(lambda x: x ** 0.5)(outputs)



  d1=DownBlock(input_img,7,128,3, True,12)
  print("D1")
  d2=DownBlock(d1,128,256)
  print("D2")
  d3=DownBlock(d2,256,512)
  print("D3")
  d4=DownBlock(d3,512,1024)
  print("D4")
  print(d4)
  u1=UpBlock(d4,d3,1024,512)
  print("U1")
  u2=UpBlock(u1,d2,512,256,3,True,12)
  print("U2")
  u3=UpBlock(u2,d1,256,128)
  print("U3")
  
  print("X")
  print(u3)
  
  outputs = Conv2DTranspose(filters=255, kernel_size=(3,3),strides=(2,2),padding="same", kernel_initializer="he_normal")(u3)
  print("Yes")
  outputs = core.Reshape((128*128,255))(outputs)
  outputs = core.Activation('softmax')(outputs)
  
  
  
  
  
    
  model = Model(inputs=[input_img], outputs=[outputs])
  print("Yes1")
  return model
	

In [272]:
input_img = Input((128, 128, 7), name='img')
model = get_unet(input_img, n_filters=7, dropout=0.15, batchnorm=True)
#model = get_unet()

model.compile(optimizer=Adam(lr=0.1), loss="categorical_crossentropy", metrics=["categorical_accuracy"])
model.summary()

D1
D2
D3
D4
Tensor("add_200/add:0", shape=(?, 8, 8, 1024), dtype=float32)
Tensor("conv2d_transpose_78/BiasAdd:0", shape=(?, ?, ?, 512), dtype=float32)
Tensor("add_199/add:0", shape=(?, 16, 16, 512), dtype=float32)
U1
Tensor("conv2d_transpose_79/BiasAdd:0", shape=(?, ?, ?, 256), dtype=float32)
Tensor("add_198/add:0", shape=(?, 32, 32, 256), dtype=float32)
U2
Tensor("conv2d_transpose_80/BiasAdd:0", shape=(?, ?, ?, 128), dtype=float32)
Tensor("add_197/add:0", shape=(?, 64, 64, 128), dtype=float32)
U3
X
Tensor("leaky_re_lu_941/LeakyRelu:0", shape=(?, 64, 64, 128), dtype=float32)
Yes
Yes1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                (None, 128, 128, 7)  0                                            
__________________________________________________________________________________________________
conv2d_1048 (

In [0]:
callbacks = [
    EarlyStopping(monitor='loss',patience=10, verbose=1),
    ReduceLROnPlateau(monitor='loss',factor=0.1, patience=3, min_lr=0.00000001, verbose=1),
    ModelCheckpoint('/content/drive/My Drive/model3.h5', monitor='loss',verbose=1, save_best_only=True, save_weights_only=False)
]



In [0]:
train_generator = get_data(train_ids, batch_size=2)
valid_generator = get_data(valid_ids,batch_size=2)


In [0]:
#new_model.predict_generator(train_generator,steps=2,verbose=1)
results =model.fit_generator(train_generator, steps_per_epoch=675 , epochs=500, verbose=1, validation_data=valid_generator, validation_steps=1, callbacks=callbacks)

In [0]:
model.evaluate_generator(get_data(train_ids, batch_size=1),steps=375,verbose=1)

In [0]:
%matplotlib inline
from matplotlib import pyplot as plt

X_test = np.zeros((1, im_height, im_width, 7), dtype=np.float32)

for r in range(1,8):
          img = load_img(path_train_input + 'lc8' + valid_ids[50][3:-4] + '_' + str(r) + '.tif', color_mode="grayscale")
          plt.imshow(img)
          plt.show()
          x_img = img_to_array(img)
          x_img = resize(x_img, (128, 128), mode='constant', preserve_range=True)
          for p in range(128):
            for q in range(128):
              X_test[0][p][q][r-1]=x_img[p][q]/255
              
           



In [0]:
print(X_test.shape)

In [0]:


ans=model.predict(X_test)
ans=ans.reshape(1,128,128,255)


ans_final=np.zeros((128,128,1))
index=-1
maxim=-1

for i in range(128):
  for j in range(128):
    for l in range(255):
      if(ans[0][i][j][l]>maxim):
        maxim=ans[0][i][j][l]
        index=l
    ans_final[i][j][0]=index
    index=-1
    maxim=-1
    
    
print(ans_final)

In [0]:



img = array_to_img(ans_final)

plt.imshow(img)
plt.show()

print(ans_final)

true_img=load_img(path_train_output+valid_ids[50],color_mode="grayscale")
plt.imshow(true_img)
plt.show()

print(img_to_array(true_img))